In [1]:
import urllib
from IPython.display import Image, display, clear_output
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

import json
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
sns.set_style('whitegrid')


In [2]:
import os
import h5py
import numpy as np
import pandas as pd
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.regularizers import l2,l1
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.callbacks import ModelCheckpoint, History


C:\Users\ssasi\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras import backend as K
K.set_image_dim_ordering('th')



In [4]:
img_width = 224 
img_height = 224

In [5]:
from keras.models import load_model
model1 = load_model('vgg16_1.h5')

C:\Users\ssasi\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
model1.layers.pop()
model1.layers.pop()
model1.layers.pop()
model1.layers.pop()

In [7]:
def save_bottleneck_features(location):
    datagen = ImageDataGenerator(rescale=1./255)  
    
    model = model1
    
    generator = datagen.flow_from_directory('C:\\Users\\ssasi\\Downloads\\data1a\\training',
                                            target_size=(img_width, img_height),
                                            batch_size=16, 
                                            class_mode=None, 
                                            shuffle=False) 
    print(generator)
    
    bottleneck_features_train = model.predict_generator(generator, 1840)
    np.save(open(location+'/bottleneck_features_train.npy', 'w'), bottleneck_features_train)
    
    # repeat with the validation data
    generator = datagen.flow_from_directory('C:\\Users\\ssasi\\Downloads\\data1a\\validation',
                                           target_size=(img_width, img_height),
                                           batch_size=16,
                                           class_mode=None,
                                           shuffle=False)
    bottleneck_features_validation = model.predict_generator(generator, 460)
    np.save(open(location+'/bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)

In [8]:
location = 'data1a'

In [14]:
datagen = ImageDataGenerator(rescale=1./255)  
    
model = model1
    
generator = datagen.flow_from_directory('C:\\Users\\ssasi\\Downloads\\data1a\\training',
                                            target_size=(img_width, img_height),
                                            batch_size=16, 
                                            class_mode=None, 
                                            shuffle=False) 
# print(generator)
    
bottleneck_features_train = model.predict_generator(generator, 3)
np.save(open(location+'/bottleneck_features_train.npy', 'wb'), bottleneck_features_train)


Found 1840 images belonging to 2 classes.


In [15]:
type(bottleneck_features_train)

numpy.ndarray

### Don't run the below code.........

In [9]:
save_bottleneck_features(location)

Found 1840 images belonging to 2 classes.


KeyboardInterrupt: 

In [ ]:
def train_binary_model():

    train_data = np.load(open(location+'/bottleneck_features_train.npy'))
    train_labels = np.array([0] * train_samples[0] + 
                            [1] * train_samples[1])

    validation_data = np.load(open(location+'/bottleneck_features_validation.npy'))
    validation_labels = np.array([0] * validation_samples[0] + 
                                 [1] * validation_samples[1])
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:])) # 512, 4, 4
    model.add(Dense(256, activation = 'relu', W_regularizer=l2(0.01)))
    model.add(Dropout(0.5)) 
    model.add(Dense(1, activation = 'sigmoid')) # should activation be sigmoid for binary problem?

    model.compile(optimizers.SGD(lr=0.0001, momentum=0.9),
              loss='binary_crossentropy', metrics=['accuracy'])

    
    checkpoint = ModelCheckpoint(top_model_weights_path, monitor='val_acc', 
                                 verbose=1, save_best_only=True, save_weights_only=True, mode='auto')

    fit = model.fit(train_data, train_labels,
              nb_epoch=nb_epoch, batch_size=16,
              validation_data=(validation_data, validation_labels),
              callbacks=[checkpoint])
    
    with open(location+'/top_history.txt', 'wb') as f:
        json.dump(fit.history, f)
    
    return model, fit.history